In [41]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchaudio
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from PIL import Image

import pandas as pd
import json
import time
import torch.nn.functional as F
from sklearn.metrics import fbeta_score
from sklearn.utils.class_weight import compute_class_weight
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB
from torchvision.transforms.functional import to_tensor

import warnings
warnings.filterwarnings("ignore")
from google.colab import drive #เชื่อมกับ google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from lightgbm import LGBMRegressor


In [43]:
# โหลดข้อมูล
train = pd.read_csv('/content/drive/MyDrive/SPU_MiniHack/train.csv')
test = pd.read_csv('/content/drive/MyDrive/SPU_MiniHack/test.csv')

# แยก features และ target
X = train.drop(columns=['delivery_time_minutes', 'parcel_id'])
y = train['delivery_time_minutes']
X_test = test.drop(columns=['parcel_id'])

# ฟีเจอร์เชิงหมวดหมู่และเชิงตัวเลข
categorical_cols = ['zone_type', 'weather_condition', 'day_of_week']
numerical_cols = ['distance_km', 'weight_kg']

# ตัวแปลงฟีเจอร์
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

# รวม pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# สร้าง pipeline กับ XGBoost
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42))
])

# แบ่งข้อมูลสำหรับ validation (optional)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# เทรนโมเดล
model.fit(X_train, y_train)

# ประเมินค่า (optional)
y_pred_val = model.predict(X_val)
mae = mean_absolute_error(y_val, y_pred_val)
print(f"Validation MAE: {mae:.2f}")

# ทำนายข้อมูล test
test_preds = model.predict(X_test)

# สร้าง submission.csv
submission = pd.DataFrame({
    'parcel_id': test['parcel_id'],
    'delivery_time_minutes': test_preds
})

submission.to_csv('submission.csv', index=False)
print("สร้างไฟล์ submission.csv เรียบร้อยแล้ว")

Validation MAE: 6.99
สร้างไฟล์ submission.csv เรียบร้อยแล้ว


In [44]:
# โหลดข้อมูล
train = pd.read_csv('/content/drive/MyDrive/SPU_MiniHack/train.csv')
test = pd.read_csv('/content/drive/MyDrive/SPU_MiniHack/test.csv')

# เตรียมข้อมูล
X = train.drop(columns=['delivery_time_minutes', 'parcel_id'])
y = train['delivery_time_minutes']
X_test = test.drop(columns=['parcel_id'])

# ฟีเจอร์
categorical_cols = ['zone_type', 'weather_condition', 'day_of_week']
numerical_cols = ['distance_km', 'weight_kg']

# ตัวแปลงข้อมูล
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

preprocessor = ColumnTransformer([
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)
])

# แยก train/val
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

# สร้าง pipeline พร้อม XGBoost
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        early_stopping_rounds=20
    ))
])

# แปลง validation สำหรับ early stopping
X_train_proc = preprocessor.fit_transform(X_train)
X_val_proc = preprocessor.transform(X_val)

# เทรนเฉพาะ regressor เพื่อ early stopping
model.named_steps['regressor'].fit(
    X_train_proc, y_train,
    eval_set=[(X_val_proc, y_val)],
    verbose=False
)

# แทน regressor ที่เทรนแล้วใน pipeline
model.steps[-1] = ('regressor', model.named_steps['regressor'])

# ===== Evaluate บน Validation =====
y_pred_val = model.predict(X_val)
mae = mean_absolute_error(y_val, y_pred_val)

mse = mean_squared_error(y_val, y_pred_val)
rmse = np.sqrt(mse)

r2 = r2_score(y_val, y_pred_val)

# ความแม่นยำแบบกำหนดขอบเขต ±10 นาที
tolerance = 10
acc = np.mean(np.abs(y_pred_val - y_val) <= tolerance)

print(f"Validation MAE: {mae:.4f}")
print(f"Validation RMSE: {rmse:.4f}")
print(f"R² Score (Regression Accuracy): {r2:.4f}")
print(f"Custom Accuracy (±{tolerance} mins): {acc * 100:.2f}%")

# ===== ทำนาย test และสร้าง submission.csv =====
y_test_pred = model.predict(X_test)
submission = pd.DataFrame({
    'parcel_id': test['parcel_id'],
    'delivery_time_minutes': y_test_pred
})
submission.to_csv('submission.csv', index=False)
print("สร้าง submission.csv เรียบร้อยแล้ว")

Validation MAE: 7.7687
Validation RMSE: 10.8309
R² Score (Regression Accuracy): 0.9949
Custom Accuracy (±10 mins): 71.33%
สร้าง submission.csv เรียบร้อยแล้ว


In [45]:
# ===== โหลดข้อมูล =====
train = pd.read_csv('/content/drive/MyDrive/SPU_MiniHack/train.csv')
test = pd.read_csv('/content/drive/MyDrive/SPU_MiniHack/test.csv')

# ===== Feature Engineering =====
day_order = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
train['day_of_week'] = pd.Categorical(train['day_of_week'], categories=day_order, ordered=True)
test['day_of_week'] = pd.Categorical(test['day_of_week'], categories=day_order, ordered=True)

# ฟีเจอร์ใหม่
train['weight_per_km'] = train['weight_kg'] / train['distance_km']
train['is_heavy'] = (train['weight_kg'] > 10).astype(int)

test['weight_per_km'] = test['weight_kg'] / test['distance_km']
test['is_heavy'] = (test['weight_kg'] > 10).astype(int)

# เตรียมข้อมูล
X = train.drop(columns=['delivery_time_minutes', 'parcel_id'])
y = train['delivery_time_minutes']
X_test = test.drop(columns=['parcel_id'])

categorical_cols = ['zone_type', 'weather_condition', 'day_of_week']
numerical_cols = ['distance_km', 'weight_kg', 'weight_per_km', 'is_heavy']

# ตัวแปลงข้อมูล
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

numerical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

preprocessor = ColumnTransformer([
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)
])

# แยก train/val
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

# ===== สร้าง pipeline =====
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LGBMRegressor(
        n_estimators=1200,
        learning_rate=0.03,
        max_depth=7,
        num_leaves=32,
        subsample=0.85,
        colsample_bytree=0.85,
        random_state=42
    ))
])

# ===== เทรน =====
model.fit(X_train, y_train)

# ===== Evaluate บน Validation =====
y_pred_val = model.predict(X_val)
mae = mean_absolute_error(y_val, y_pred_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
r2 = r2_score(y_val, y_pred_val)

tolerance = 10
acc = np.mean(np.abs(y_pred_val - y_val) <= tolerance)

print(f"Validation MAE: {mae:.4f}")
print(f"Validation RMSE: {rmse:.4f}")
print(f"R² Score (Regression Accuracy): {r2:.4f}")
print(f"Custom Accuracy (±{tolerance} mins): {acc * 100:.2f}%")

# ===== ทำนาย test และบันทึกผล =====
y_test_pred = model.predict(X_test)
submission = pd.DataFrame({
    'parcel_id': test['parcel_id'],
    'delivery_time_minutes': y_test_pred
})
submission.to_csv('sample_submission.csv', index=False)
print("สร้าง submission.csv เรียบร้อยแล้ว")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000109 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 778
[LightGBM] [Info] Number of data points in the train set: 850, number of used features: 7
[LightGBM] [Info] Start training from score 211.330765
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [46]:
# ================== PART 1: LIGHTGBM TRAINING ==================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from lightgbm import LGBMRegressor

# Load data
train = pd.read_csv('/content/drive/MyDrive/SPU_MiniHack/train.csv')
test = pd.read_csv('/content/drive/MyDrive/SPU_MiniHack/test.csv')

# ===== Feature Engineering =====
day_order = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
train['day_of_week'] = pd.Categorical(train['day_of_week'], categories=day_order, ordered=True)
test['day_of_week'] = pd.Categorical(test['day_of_week'], categories=day_order, ordered=True)

# ฟีเจอร์ใหม่
for df in [train, test]:
    df['weight_per_km'] = df['weight_kg'] / df['distance_km']
    df['is_heavy'] = (df['weight_kg'] > 10).astype(int)
    df['zone_weather'] = df['zone_type'] + "_" + df['weather_condition']

# เตรียมข้อมูล
X = train.drop(columns=['delivery_time_minutes', 'parcel_id'])
y = train['delivery_time_minutes']
X_test = test.drop(columns=['parcel_id'])

categorical_cols = ['zone_type', 'weather_condition', 'day_of_week', 'zone_weather']
numerical_cols = ['distance_km', 'weight_kg', 'weight_per_km', 'is_heavy']

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

numerical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

preprocessor = ColumnTransformer([
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)
])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LGBMRegressor(
        n_estimators=1200,
        learning_rate=0.03,
        max_depth=7,
        num_leaves=32,
        subsample=0.85,
        colsample_bytree=0.85,
        random_state=42
    ))
])

model.fit(X_train, y_train)

# ===== Evaluate =====
y_pred_val = model.predict(X_val)
mae = mean_absolute_error(y_val, y_pred_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
r2 = r2_score(y_val, y_pred_val)
acc = np.mean(np.abs(y_pred_val - y_val) <= 10)

print(f"Validation MAE: {mae:.4f}")
print(f"Validation RMSE: {rmse:.4f}")
print(f"R² Score: {r2:.4f}")
print(f"Accuracy (±10 mins): {acc * 100:.2f}%")

# ===== ทำนาย test =====
y_test_pred = model.predict(X_test)
submission = pd.DataFrame({'parcel_id': test['parcel_id'], 'delivery_time_minutes': y_test_pred})
submission.to_csv('sample_submission.csv', index=False)
print("สร้าง sample_submission.csv เรียบร้อยแล้ว")



# ================== PART 2: CNN WITH BOTTLENECK + SEBLOCK ==================
import torch
import torch.nn as nn
import torch.nn.functional as F

# ===== Squeeze-and-Excitation Block =====
class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super().__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, in_channels // reduction, 1),
            nn.ReLU(),
            nn.Conv2d(in_channels // reduction, in_channels, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        weight = self.se(x)
        return x * weight


# ===== Bottleneck Residual Block =====
class BottleneckResidualBlock(nn.Module):
    def __init__(self, channels, bottleneck_ratio=4):
        super().__init__()
        bottleneck_channels = channels // bottleneck_ratio
        self.block = nn.Sequential(
            nn.Conv2d(channels, bottleneck_channels, 1),
            nn.BatchNorm2d(bottleneck_channels),
            nn.ReLU(),
            nn.Conv2d(bottleneck_channels, bottleneck_channels, 3, padding=1),
            nn.BatchNorm2d(bottleneck_channels),
            nn.ReLU(),
            nn.Conv2d(bottleneck_channels, channels, 1),
            nn.BatchNorm2d(channels)
        )
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.relu(self.block(x) + x)


# ===== Double Convolution Block =====
def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU()
    )

# ===== Custom CNN Model =====
class DeliveryCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # Layer 1
        self.conv1 = double_conv(3, 32)
        self.pool1 = nn.MaxPool2d(2)

        # Layer 2
        self.conv2 = double_conv(32, 64)
        self.pool2 = nn.MaxPool2d(2)

        # Layer 3
        self.conv3 = double_conv(64, 128)
        self.bottleneck3 = BottleneckResidualBlock(128)
        self.se3 = SEBlock(128)

        # Layer 4
        self.conv4 = double_conv(128, 256)
        self.bottleneck4 = BottleneckResidualBlock(256)
        self.se4 = SEBlock(256)

        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(256, 1)

    def forward(self, x):
        x = self.pool1(self.conv1(x))
        x = self.pool2(self.conv2(x))

        x = self.conv3(x)
        x = self.bottleneck3(x)
        x = self.se3(x)

        x = self.conv4(x)
        x = self.bottleneck4(x)
        x = self.se4(x)

        x = self.global_pool(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000123 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 787
[LightGBM] [Info] Number of data points in the train set: 850, number of used features: 8
[LightGBM] [Info] Start training from score 211.330765
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [47]:
pip install tabpfn

In [48]:
from tabpfn import TabPFNClassifier, TabPFNRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd
import numpy as np

# Load data
train = pd.read_csv('/content/drive/MyDrive/SPU_MiniHack/train.csv')
test = pd.read_csv('/content/drive/MyDrive/SPU_MiniHack/test.csv')

# Feature Engineering
order = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
for df in [train, test]:
    df['day_of_week'] = pd.Categorical(df['day_of_week'], categories=order, ordered=True)
    df['weight_per_km'] = df['weight_kg'] / df['distance_km']
    df['is_heavy'] = (df['weight_kg'] > 10).astype(int)
    df['zone_weather'] = df['zone_type'] + "_" + df['weather_condition']

X = train.drop(columns=['delivery_time_minutes', 'parcel_id'])
y = train['delivery_time_minutes']
X_test = test.drop(columns=['parcel_id'])

categorical_cols = ['zone_type', 'weather_condition', 'day_of_week', 'zone_weather']
numerical_cols = ['distance_km', 'weight_kg', 'weight_per_km', 'is_heavy']

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

numerical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

preprocessor = ColumnTransformer([
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)
])

X_processed = preprocessor.fit_transform(X)
X_test_processed = preprocessor.transform(X_test)

X_train, X_val, y_train, y_val = train_test_split(X_processed, y, test_size=0.15, random_state=42)

# ใช้ TabPFN Regressor
model = TabPFNRegressor(device="cuda" if torch.cuda.is_available() else "cpu")
model.fit(X_train, y_train)

# Evaluate
y_pred_val = model.predict(X_val)
mae = mean_absolute_error(y_val, y_pred_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
r2 = r2_score(y_val, y_pred_val)
acc = np.mean(np.abs(y_pred_val - y_val) <= 10)

print(f"[TabPFN] MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}, Accuracy (±10 mins): {acc * 100:.2f}%")

# Predict test set
y_test_pred = model.predict(X_test_processed)
submission = pd.DataFrame({'parcel_id': test['parcel_id'], 'delivery_time_minutes': y_test_pred})
submission.to_csv('sample_submission.csv', index=False)
print("สร้าง sample_submission.csv เรียบร้อยแล้ว")

[TabPFN] MAE: 4.3786, RMSE: 5.5455, R2: 0.9987, Accuracy (±10 mins): 95.33%
สร้าง sample_submission.csv เรียบร้อยแล้ว
